# Spaceship Titanic - V4 (V2 Base + Surgical EDA Features)

**V1:** CV 0.8265 / LB 0.8020 (overfitting, 49 features)  
**V2:** CV 0.8141 / LB 0.8071 (clean, 29 features, gap 0.007)  
**V3:** CV 0.8184 / LB 0.8041 (overfit, 56 features, target encoding)  
**V4 Goal:** V2 base + ONLY simple, robust EDA features  

**What went wrong in V3:**
- Target encoding on 8700 rows = overfit even with CV-safe
- Too many interaction label encodings (high cardinality)
- 56 features = noise > signal ratio too high
- Slightly weaker regularization than V2

**V4 changes from V2:**
- Smart imputation from EDA (HomePlanet←Deck mapping)
- +IsBaby (81% transported), +Awake_NoSpend (62% transported)
- +n_missing (data quality signal)
- +DeckSide_freq ONLY (frequency, NOT target/label encoding)
- Same strong regularization as V2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
import warnings, os

warnings.filterwarnings('ignore')

SEED = 42
N_FOLDS = 10
TARGET = 'Transported'

def seed_everything(seed=SEED):
    import random
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything()
print('V4 Setup complete.')

In [ ]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
sample_sub = pd.read_csv('../data/sample_submission.csv')

# Save original for n_missing computation
train_orig_nulls = train.isnull().sum(axis=1)
test_orig_nulls = test.isnull().sum(axis=1)

train['is_train'] = 1
test['is_train'] = 0
test[TARGET] = np.nan
df = pd.concat([train, test], axis=0, ignore_index=True)

# n_missing from original data (before any imputation)
df['n_missing'] = pd.concat([train_orig_nulls, test_orig_nulls], ignore_index=True)

spend_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
print(f'Combined: {df.shape}')

## Feature Engineering V4 - V2 Base + Surgical Additions

In [ ]:
# === STEP 1: Parse raw columns (same as V2) ===
df['_Group'] = df['PassengerId'].str.split('_').str[0].astype(int)
df['GroupSize'] = df.groupby('_Group')['PassengerId'].transform('count')
df['IsAlone'] = (df['GroupSize'] == 1).astype(int)

# Cabin
df['Deck'] = df['Cabin'].str.split('/').str[0]
df['CabinNum'] = df['Cabin'].str.split('/').str[1].astype(float)
df['Side'] = df['Cabin'].str.split('/').str[2]
df['CabinRegion'] = (df['CabinNum'] // 100).astype(float)

# Name
df['Surname'] = df['Name'].str.split().str[-1]
df['FamilySize'] = df.groupby('Surname')['PassengerId'].transform('count')
df.loc[df['Surname'].isna(), 'FamilySize'] = 1

# Booleans
df['CryoSleep'] = df['CryoSleep'].map({True: 1, False: 0, 'True': 1, 'False': 0})
df['VIP'] = df['VIP'].map({True: 1, False: 0, 'True': 1, 'False': 0})

print('Step 1 done: parsing')

In [ ]:
# === STEP 2: Smart imputation (IMPROVED from EDA) ===

# CryoSleep passengers have 0 spending
for col in spend_cols:
    mask = (df['CryoSleep'] == 1) & (df[col].isna())
    df.loc[mask, col] = 0

# Infer CryoSleep from spending
total_spend_raw = df[spend_cols].sum(axis=1)
df.loc[(df['CryoSleep'].isna()) & (total_spend_raw == 0), 'CryoSleep'] = 1
df.loc[(df['CryoSleep'].isna()) & (total_spend_raw > 0), 'CryoSleep'] = 0

# NEW in V4: HomePlanet inferred from Deck (EDA: deterministic mapping)
deck_to_planet = {
    'A': 'Europa', 'B': 'Europa', 'C': 'Europa', 'T': 'Europa',
    'G': 'Earth'
}
for deck, planet in deck_to_planet.items():
    mask = (df['HomePlanet'].isna()) & (df['Deck'] == deck)
    df.loc[mask, 'HomePlanet'] = planet

# Fill remaining
for col in df.select_dtypes(include=[np.number]).columns:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].median())

for col in df.select_dtypes(include=['object']).columns:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].mode()[0])

print(f'Step 2 done: smart imputation. Nulls: {df.isnull().sum().sum()}')

In [ ]:
# === STEP 3: Spending features (V2 base) ===
df['TotalSpend'] = df[spend_cols].sum(axis=1)
df['TotalSpend_log'] = np.log1p(df['TotalSpend'])
df['NoSpend'] = (df['TotalSpend'] == 0).astype(int)
df['NumServicesUsed'] = (df[spend_cols] > 0).sum(axis=1)

for col in spend_cols:
    df[f'{col}_log'] = np.log1p(df[col])

# Luxury vs basic (V2)
df['LuxurySpend'] = np.log1p(df['Spa'] + df['VRDeck'] + df['RoomService'])
df['BasicSpend'] = np.log1p(df['FoodCourt'] + df['ShoppingMall'])

# NEW V4: Awake_NoSpend (EDA: 62% transported)
df['Awake_NoSpend'] = ((df['CryoSleep'] == 0) & (df['TotalSpend'] == 0)).astype(int)

print('Step 3 done: spending features')

In [ ]:
# === STEP 4: Age features (V2 base + 1 EDA addition) ===
df['AgeGroup'] = pd.cut(df['Age'], bins=[0, 5, 12, 18, 30, 50, 80],
                        labels=[0, 1, 2, 3, 4, 5]).astype(float)
df['IsChild'] = (df['Age'] < 18).astype(float)

# NEW V4: IsBaby (EDA: 81% transported - strong signal)
df['IsBaby'] = (df['Age'] <= 4).astype(float)

print('Step 4 done: age features')

In [ ]:
# === STEP 5: Interaction (V2) ===
df['CryoSleep_NoSpend'] = ((df['CryoSleep'] == 1) & (df['TotalSpend'] == 0)).astype(int)

# === STEP 6: Group-level spending (V2) ===
df['GroupSpend_mean'] = df.groupby('_Group')['TotalSpend'].transform('mean')
df['GroupSpend_mean_log'] = np.log1p(df['GroupSpend_mean'])

print('Step 5-6 done: interactions & group features')

In [ ]:
# === STEP 7: Encode categoricals (V2 base) ===
for col in ['HomePlanet', 'Destination', 'Deck', 'Side']:
    le = LabelEncoder()
    df[col + '_le'] = le.fit_transform(df[col].astype(str))

for col in ['HomePlanet', 'Destination', 'Deck', 'Side']:
    freq = df[col].value_counts(normalize=True)
    df[col + '_freq'] = df[col].map(freq)

# NEW V4: DeckSide frequency ONLY (no label/target encoding)
df['DeckSide'] = df['Deck'] + '_' + df['Side']
freq = df['DeckSide'].value_counts(normalize=True)
df['DeckSide_freq'] = df['DeckSide'].map(freq)

print('Step 7 done: encoding')

In [ ]:
# === DEFINE FEATURES ===
drop_cols = [
    'PassengerId', 'Name', 'Cabin', 'Surname', 'is_train', TARGET,
    'HomePlanet', 'Destination', 'Deck', 'Side', 'DeckSide',
    '_Group', 'CabinNum',
    'TotalSpend', 'GroupSpend_mean',
] + spend_cols

features = [c for c in df.columns if c not in drop_cols]
print(f'V4 features: {len(features)}')
print()
for i, f in enumerate(sorted(features)):
    print(f'  {i+1:2d}. {f}')

In [ ]:
# Split
train_df = df[df['is_train'] == 1].copy()
test_df = df[df['is_train'] == 0].copy()

X = train_df[features].values
y = train_df[TARGET].astype(int).values
X_test = test_df[features].values

print(f'X: {X.shape}, y: {y.shape}, X_test: {X_test.shape}')
print(f'Target mean: {np.mean(y):.4f}')

## LightGBM V4 (V2 regularization)

In [ ]:
lgb_params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 20,
    'learning_rate': 0.03,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.7,
    'bagging_freq': 5,
    'min_child_samples': 30,
    'reg_alpha': 0.5,
    'reg_lambda': 2.0,
    'max_depth': 5,
    'n_estimators': 5000,
    'verbose': -1,
    'n_jobs': -1,
    'random_state': SEED,
}

oof_lgb = np.zeros(len(X))
test_lgb = np.zeros(len(X_test))
fi_lgb = np.zeros(len(features))

kf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
    X_tr, X_val = X[train_idx], X[val_idx]
    y_tr, y_val = y[train_idx], y[val_idx]
    
    model = lgb.LGBMClassifier(**lgb_params)
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        callbacks=[lgb.early_stopping(200), lgb.log_evaluation(500)]
    )
    
    oof_lgb[val_idx] = model.predict_proba(X_val)[:, 1]
    test_lgb += model.predict_proba(X_test)[:, 1] / N_FOLDS
    fi_lgb += model.feature_importances_ / N_FOLDS
    
    fold_acc = accuracy_score(y_val, (oof_lgb[val_idx] > 0.5).astype(int))
    print(f'Fold {fold+1}/{N_FOLDS} - Accuracy: {fold_acc:.5f}')

lgb_acc = accuracy_score(y, (oof_lgb > 0.5).astype(int))
print(f'\nLightGBM V4 CV: {lgb_acc:.5f}')

In [ ]:
# Feature importance
fi_df = pd.DataFrame({'feature': features, 'importance': fi_lgb})
fi_df = fi_df.sort_values('importance', ascending=True).tail(20)

plt.figure(figsize=(10, 7))
plt.barh(fi_df['feature'], fi_df['importance'], color='steelblue')
plt.title(f'LightGBM V4 Feature Importance - CV: {lgb_acc:.5f}')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

## XGBoost V4 (V2 regularization)

In [ ]:
xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 4,
    'learning_rate': 0.03,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'min_child_weight': 10,
    'reg_alpha': 0.5,
    'reg_lambda': 2.0,
    'gamma': 0.1,
    'n_estimators': 5000,
    'early_stopping_rounds': 200,
    'tree_method': 'hist',
    'random_state': SEED,
    'verbosity': 0,
}

oof_xgb = np.zeros(len(X))
test_xgb = np.zeros(len(X_test))

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
    X_tr, X_val = X[train_idx], X[val_idx]
    y_tr, y_val = y[train_idx], y[val_idx]
    
    model = xgb.XGBClassifier(**xgb_params)
    model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=500)
    
    oof_xgb[val_idx] = model.predict_proba(X_val)[:, 1]
    test_xgb += model.predict_proba(X_test)[:, 1] / N_FOLDS
    
    fold_acc = accuracy_score(y_val, (oof_xgb[val_idx] > 0.5).astype(int))
    print(f'Fold {fold+1}/{N_FOLDS} - Accuracy: {fold_acc:.5f}')

xgb_acc = accuracy_score(y, (oof_xgb > 0.5).astype(int))
print(f'\nXGBoost V4 CV: {xgb_acc:.5f}')

## CatBoost V4 (V2 regularization)

In [ ]:
oof_cb = np.zeros(len(X))
test_cb = np.zeros(len(X_test))

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
    X_tr, X_val = X[train_idx], X[val_idx]
    y_tr, y_val = y[train_idx], y[val_idx]
    
    model = CatBoostClassifier(
        iterations=5000,
        learning_rate=0.03,
        depth=4,
        l2_leaf_reg=5.0,
        subsample=0.7,
        colsample_bylevel=0.7,
        min_data_in_leaf=30,
        random_seed=SEED,
        verbose=500,
        early_stopping_rounds=200,
        task_type='CPU',
    )
    
    model.fit(X_tr, y_tr, eval_set=(X_val, y_val))
    
    oof_cb[val_idx] = model.predict_proba(X_val)[:, 1]
    test_cb += model.predict_proba(X_test)[:, 1] / N_FOLDS
    
    fold_acc = accuracy_score(y_val, (oof_cb[val_idx] > 0.5).astype(int))
    print(f'Fold {fold+1}/{N_FOLDS} - Accuracy: {fold_acc:.5f}')

cb_acc = accuracy_score(y, (oof_cb > 0.5).astype(int))
print(f'\nCatBoost V4 CV: {cb_acc:.5f}')

## Ensemble & Submission

In [ ]:
print('=== V4 Results ===')
print(f'LightGBM: {lgb_acc:.5f}')
print(f'XGBoost:  {xgb_acc:.5f}')
print(f'CatBoost: {cb_acc:.5f}')

# Simple average
oof_avg = (oof_lgb + oof_xgb + oof_cb) / 3
avg_acc = accuracy_score(y, (oof_avg > 0.5).astype(int))
print(f'\nSimple Average: {avg_acc:.5f}')

# Majority voting
votes = ((oof_lgb > 0.5).astype(int) + (oof_xgb > 0.5).astype(int) + (oof_cb > 0.5).astype(int))
vote_acc = accuracy_score(y, (votes >= 2).astype(int))
print(f'Majority Voting: {vote_acc:.5f}')

# Use simple average for submission
final_proba = (test_lgb + test_xgb + test_cb) / 3
final_preds = (final_proba > 0.5)

print(f'\nTest: {final_preds.sum()} True / {len(final_preds) - final_preds.sum()} False')
print(f'Ratio: {final_preds.mean():.4f}')

print(f'\n=== VERSION COMPARISON ===')
print(f'V1: CV 0.82653 | LB 0.80196 | Gap 0.0246 | 49 features')
print(f'V2: CV 0.81410 | LB 0.80710 | Gap 0.0070 | 29 features')
print(f'V3: CV 0.81836 | LB 0.80406 | Gap 0.0143 | 56 features')
print(f'V4: CV {avg_acc:.5f} | LB TBD     |            | {len(features)} features')

In [ ]:
submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'].values,
    'Transported': final_preds
})
submission['Transported'] = submission['Transported'].astype(bool)
submission.to_csv('../submissions/submission_v4.csv', index=False)

assert submission.shape[0] == sample_sub.shape[0]
assert list(submission.columns) == list(sample_sub.columns)
assert submission['Transported'].dtype == bool

print('V4 Submission saved: submissions/submission_v4.csv')
print(submission['Transported'].value_counts(normalize=True))
submission.head()